In [1]:
!nvidia-smi

Mon Aug 31 01:30:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import files, drive
!mkdir -p drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pip install transformers
!pip install pytorch-crf==0.7.2  
!pip install tensorboardX

     |████████████████████████████████| 778kB 10.8MB/s 
     |████████████████████████████████| 1.1MB 46.9MB/s 
     |████████████████████████████████| 3.0MB 52.6MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a0be146db106016c7f4dcb47f3af4e66bc94caf767914d835b03c02381efe893
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 317kB 11.5MB/s 


In [4]:
import logging
import os
import sys
import numpy as np
from collections import defaultdict, namedtuple
import re

import csv
import random
import json
import pickle
import codecs


import argparse

import datetime
import time


from tqdm import tqdm,tqdm_notebook, trange


import torch
import torch.nn as nn
import torch.functional as F
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.utils.data.distributed import DistributedSampler
from torchcrf import CRF


from tensorboardX import SummaryWriter


from transformers import AutoModelForPreTraining, AutoModel,AutoConfig,AutoTokenizer,AutoModelForTokenClassification, BertForPreTraining,BertModel,BertTokenizer,BertConfig
from transformers import WEIGHTS_NAME 
from transformers import AdamW, get_linear_schedule_with_warmup

In [5]:

master_holder='/content/gdrive/My Drive/ETNET/NER/news/code/PeijiYang'#


In [6]:

#model_name='electra-hongkongese-large-discriminator'
model_name='bert-base-multilingual-cased'
pretrained_model_path = os.path.join(master_holder,'pretrain_models',model_name)

data_dir=os.path.join(master_holder,'dataset')
cache_dir=os.path.join(master_holder,'cache')
output_dir=os.path.join(master_holder,'checkpoints_bert')

In [7]:
train_file=os.path.join(data_dir,'train.txt')
eval_file=os.path.join(data_dir,'dev.txt')


In [8]:
def boolean_string(s):
    if s not in {'False', 'True'}:
        raise ValueError('Not a valid boolean string')
    return s == 'True'

In [9]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [10]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 7431, done.
remote: Total 7431 (delta 0), reused 0 (delta 0), pack-reused 7431
Receiving objects: 100% (7431/7431), 13.90 MiB | 5.20 MiB/s, done.
Resolving deltas: 100% (5024/5024), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-oq0n723q
Created temporary directory: /tmp/pip-req-tracker-otckqyrh
Created requirements tracker '/tmp/pip-req-tracker-otckqyrh'
Created temporary directory: /tmp/pip-install-tqjsbktm
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-6u8xt6t8
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-otckqyrh'
    Running setup.py (path:/tmp/pip-req-build-6u8xt6t8/setup.py) egg_info for package from file:///content/apex
    Running

In [11]:
try:
    from apex import amp
except ImportError:
    raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

In [12]:

###  arguments

parser = argparse.ArgumentParser()

## Required parameters
parser.add_argument("--train_file", default=None, type=str)
parser.add_argument("--eval_file", default=None, type=str)
parser.add_argument("--test_file", default=None, type=str)
parser.add_argument("--model_name_or_path", default=None, type=str)
parser.add_argument("--output_dir", default=None, type=str)


## other parameters
parser.add_argument("--config_name", default="", type=str,  help="Pretrained config name or path if not the same as model_name")
parser.add_argument("--tokenizer_name", default="", type=str,  help="Pretrained tokenizer name or path if not the same as model_name")
parser.add_argument("--cache_dir", default="", type=str,  help="Where do you want to store the pre-trained models downloaded from s3")
    
parser.add_argument("--max_seq_length", default=128, type=int)
parser.add_argument("--do_train", default=False, type=boolean_string)
parser.add_argument("--do_eval", default=False, type=boolean_string)
parser.add_argument("--do_test", default=False, type=boolean_string)

parser.add_argument("--train_batch_size", default=24, type=int)
parser.add_argument("--eval_batch_size", default=24, type=int)

parser.add_argument("--learning_rate", default=3e-5, type=float)
parser.add_argument("--num_train_epochs", default=30, type=float)
parser.add_argument("--warmup_proprotion", default=0.1, type=float)
parser.add_argument("--use_weight", default=1, type=int)
parser.add_argument("--local_rank", type=int, default=-1)
parser.add_argument("--seed", type=int, default=42)
parser.add_argument("--fp16", default=False)
parser.add_argument('--fp16_opt_level', type=str, default='O1', help="For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3'].See details at https://nvidia.github.io/apex/amp.html")
parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
parser.add_argument("--loss_scale", type=float, default=0)
parser.add_argument('--gradient_accumulation_steps', type=int, default=1)
parser.add_argument("--warmup_steps", default=0, type=int)
parser.add_argument("--adam_epsilon", default=1e-8, type=float)
parser.add_argument("--max_steps", default=-1, type=int)

parser.add_argument("--do_lower_case", action='store_true')
parser.add_argument("--logging_steps", default=500, type=int)
parser.add_argument("--clean", default=False, type=boolean_string, help="clean the output dir")

parser.add_argument("--need_birnn", default=False, type=boolean_string)
parser.add_argument("--rnn_dim", default=128, type=int)



_StoreAction(option_strings=['--rnn_dim'], dest='rnn_dim', nargs=None, const=None, default=128, type=<class 'int'>, choices=None, help=None, metavar=None)

In [13]:
args = parser.parse_args(["--train_file",train_file ,
  "--eval_file",eval_file ,
  
  "--model_name_or_path",pretrained_model_path ,
  "--output_dir", output_dir,
  "--cache_dir", cache_dir, 
  "--do_train", 'True',
  "--do_eval", 'True' ,
  "--do_test", 'False' ,
  "--max_seq_length" ,'128',
   
  "--train_batch_size", '132' ,
  "--eval_batch_size", '132' ,
  "--num_train_epochs", '50' ,
  
  "--logging_steps", '1000' ,
  "--need_birnn", 'True' ,
  "--rnn_dim", '512' ,
  "--clean" ,'False' ,                   
])

In [14]:

## check output folder 

if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
  print("Output directory ({}) already exists and is not empty.".format(args.output_dir))
  

Output directory (/content/gdrive/My Drive/ETNET/NER/news/code/PeijiYang/checkpoints_bert) already exists and is not empty.


In [15]:

if not os.path.exists(args.output_dir):
  os.makedirs(args.output_dir)

if not os.path.exists(os.path.join(args.output_dir, "eval")):
  os.makedirs(os.path.join(args.output_dir, "eval"))

In [16]:

## check SummaryWriter 
writer = SummaryWriter(logdir=os.path.join(args.output_dir, "eval"), comment="Linear")

In [17]:
### define device :cpu or cuda
device = torch.device("cuda")
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
args.device = device
n_gpu = torch.cuda.device_count()

print('device: ',device)
print('n_gpu: ',torch.cuda.device_count())

device:  cuda
n_gpu:  1


In [18]:
### define gradient_accumulation_steps

if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(  args.gradient_accumulation_steps))

In [ ]:


### 1. utils.py
### 2. clue_process.py

### 3. models.py

### 4. ner.py
### 5. conlleval.py (optional)

In [19]:
### 1. utils.py
### 1.1 utils.py InputExample, InputFeatures
class InputExample(object):##### """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text, label=None):
        self.guid = guid
        self.text = text 
        self.label = label

class InputFeatures(object):### """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, ori_tokens):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.ori_tokens = ori_tokens

In [20]:
### 1. utils.py
### 1.2 utils.py NerProcessor

class NerProcessor(object):
    def read_data(self, input_file):#### """Reads a BIO data."""
        with open(input_file, "r", encoding="utf-8") as f:

            lines,words,labels = [],[],[]
            
            for line in tqdm_notebook(f.readlines()):   
                contends = line.strip().strip('\n')
                tokens = line.strip().strip('\n').split(" ")
                try:
                  if len(tokens) == 2:
                    words.append(tokens[0].strip())
                    labels.append(tokens[1].strip())
                  else:
                    if len(contends) == 0 and len(words) > 0:
                        label,word = [],[]
                        for l, w in zip(labels, words):
                            if len(l) > 0 and len(w) > 0:
                                label.append(l)
                                word.append(w)
                        lines.append([' '.join(label), ' '.join(word)])
                        words,labels = [],[]
                except:
                  print('Cannot process in read_data in NERProcessor of util.py',line)
                          
            
            return lines
    
    def get_labels(self, args):
        labels = set()
        if os.path.exists(os.path.join(args.output_dir, "label_list.pkl")):
            print(f"loading labels info from {args.output_dir}")
            with open(os.path.join(args.output_dir, "label_list.pkl"), "rb") as f:
                 labels = pickle.load(f)
                 
        else:
            # get labels from train data
            print(f"loading labels info from train file and dump in {args.output_dir}")
            with open(args.train_file) as f:
                for line in f.readlines():
                    tokens = line.strip().strip('\n').split(" ")

                    if len(tokens) == 2:
                        labels.add(tokens[1])

            if len(labels) > 0:
                with open(os.path.join(args.output_dir, "label_list.pkl"), "wb") as f:
                    pickle.dump(labels, f)
            else:
                print("loading error and return the default labels B,I,O")
                labels = {"O", "B", "I"}
        
        return labels 

    def get_examples(self, input_file):

        examples = []
        
        lines = self.read_data(input_file)

        for i, line in tqdm_notebook(enumerate(lines)):
            
            guid ,label,text = str(i),line[0],line[1]
            
            examples.append(InputExample(guid=guid, text=text, label=label))
        
        return examples


In [21]:
### 1. utils.py
### 1.3 utils.py convert_examples_to_features

def convert_examples_to_features(args, examples, label_list, max_seq_length, tokenizer):

    #label_map = {label : i for i, label in enumerate(label_list)}
    label_map=args.label2id
    features = []

    for (ex_index, example) in tqdm(enumerate(examples), desc="convert examples"):
        # if ex_index % 10000 == 0:
        #     print"Writing example %d of %d" % (ex_index, len(examples)))

      textlist = example.text.split(" ")
      labellist = example.label.split(" ")
      assert len(textlist) == len(labellist)

      tokens,labels,ori_tokens = [],[],[]

      for i, word in enumerate(textlist):
            # 防止wordPiece情况出现，不过貌似不会
            
        token = tokenizer.tokenize(word)

        tokens.extend(token)
        label_1 = labellist[i]
        ori_tokens.append(word)

        # 单个字符不会出现wordPiece
        for m in range(len(token)):
          if m == 0:

            labels.append(label_1)
          else:
            print(token,'token is longer than 1')
            if label_1 == "O":
              labels.append("O")
            else:
              labels.append("I")


      if len(tokens) >= max_seq_length - 1:
            tokens = tokens[0:(max_seq_length - 2)]  # -2 的原因是因为序列需要加一个句首和句尾标志
            labels = labels[0:(max_seq_length - 2)]
            ori_tokens = ori_tokens[0:(max_seq_length - 2)]

      ori_tokens = ["[CLS]"] + ori_tokens + ["[SEP]"]
        

      ntokens ,segment_ids ,label_ids = [],[],[]
      ntokens.append("[CLS]")
      segment_ids.append(0)
      label_ids.append(label_map["O"])

      for i, token in enumerate(tokens):
            ntokens.append(token)
            segment_ids.append(0)
            label_ids.append(label_map[labels[i]])

      ntokens.append("[SEP]")


      segment_ids.append(0)
      label_ids.append(label_map["O"])
      input_ids = tokenizer.convert_tokens_to_ids(ntokens)   
        
      input_mask = [1] * len(input_ids)

    
      #assert len(ori_tokens) == len(ntokens), f"{len(ori_tokens)}, {len(ntokens)}, ori_tokens: {ori_tokens}, ntokens:{ntokens}"
    

      while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            # we don't concerned about it!
            label_ids.append(0)
            ntokens.append("**NULL**")

      assert len(input_ids) == max_seq_length
      assert len(input_mask) == max_seq_length
      assert len(segment_ids) == max_seq_length
      assert len(label_ids) == max_seq_length

      if ex_index < 5:
            print("*** Example ***")
            print("guid: %s" % (example.guid))
            print("tokens: %s" % " ".join([str(x) for x in ntokens]))
            print("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            print("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            print("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            print("labels: %s" % " ".join([str(x) for x in labels]))
            print("label_ids: %s" % " ".join([str(x) for x in label_ids]))
      features.append( InputFeatures(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label_ids, ori_tokens=ori_tokens))
        
    return features

In [22]:
### 1. utils.py
### 1.3 utils.py  get_Dataset

def get_Dataset(args, processor, tokenizer, mode="train"):
    if mode == "train":
        filepath = args.train_file
    elif mode == "eval":
        filepath = args.eval_file
    elif mode == "test":
        filepath = args.test_file
    else:
        raise ValueError("mode must be one of train, eval, or test")

    examples = processor.get_examples(filepath)
    label_list = args.label_list
    features = convert_examples_to_features(args, examples, label_list, args.max_seq_length, tokenizer)

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    return examples, features, data

In [23]:
####  models.py 

class BERT_BiLSTM_CRF(BertForPreTraining):

    def __init__(self, config, need_birnn=False, rnn_dim=512):
        super(BERT_BiLSTM_CRF, self).__init__(config)
        
        self.num_tags = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        out_dim = config.hidden_size
        self.need_birnn = need_birnn

        # 如果为False，则不要BiLSTM层
        if need_birnn:
            self.birnn = nn.LSTM(config.hidden_size, rnn_dim, num_layers=1, bidirectional=True, batch_first=True)
            out_dim = rnn_dim * 2
        
        self.hidden2tag = nn.Linear(out_dim, config.num_labels)
        self.crf = CRF(config.num_labels, batch_first=True)
    

    def forward(self, input_ids, tags, token_type_ids=None, input_mask=None):

        emissions = self.tag_outputs(input_ids, token_type_ids, input_mask)

        loss = -1 * self.crf(emissions, tags, mask=input_mask.byte())

        return loss

    
    def tag_outputs(self, input_ids, token_type_ids=None, input_mask=None):

        outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=input_mask)

        sequence_output = outputs[0]
        
        if self.need_birnn:
            sequence_output, _ = self.birnn(sequence_output)

        sequence_output = self.dropout(sequence_output)

        emissions = self.hidden2tag(sequence_output)

        return emissions
    

    def predict(self, input_ids, token_type_ids=None, input_mask=None):

        emissions = self.tag_outputs(input_ids, token_type_ids, input_mask)

        return self.crf.decode(emissions, input_mask.byte())

In [24]:
## 3. conlleval.py
## 3.1 conlleval.py FormatError, EvalCounts, parse_args, parse_tag

ANY_SPACE = '<SPACE>'


class FormatError(Exception):
    pass

Metrics = namedtuple('Metrics', 'tp fp fn prec rec fscore')


class EvalCounts(object):
    def __init__(self):
        self.correct_chunk = 0    # number of correctly identified chunks
        self.correct_tags = 0     # number of correct chunk tags
        self.found_correct = 0    # number of chunks in corpus
        self.found_guessed = 0    # number of identified chunks
        self.token_counter = 0    # token counter (ignores sentence breaks)

        # counts by type
        self.t_correct_chunk = defaultdict(int)
        self.t_found_correct = defaultdict(int)
        self.t_found_guessed = defaultdict(int)


def parse_args(argv):
    import argparse
    parser = argparse.ArgumentParser(  description='evaluate tagging results using CoNLL criteria', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    arg = parser.add_argument
    arg('-b', '--boundary', metavar='STR', default='-X-',  help='sentence boundary')
    arg('-d', '--delimiter', metavar='CHAR', default=ANY_SPACE,  help='character delimiting items in input')
    arg('-o', '--otag', metavar='CHAR', default='O',  help='alternative outside tag')
    arg('file', nargs='?', default=None)
    return parser.parse_args(argv)


def parse_tag(t):
    m = re.match(r'^([^-]*)-(.*)$', t)
    return m.groups() if m else (t, '')



In [25]:
## 3. conlleval.py
## 3.2 conlleval.py evaluate


def evaluate_(iterable, options=None):
    if options is None:
        options = parse_args([])    # use defaults

    counts = EvalCounts()
    num_features = None       # number of features per line
    in_correct = False        # currently processed chunks is correct until now
    last_correct = 'O'        # previous chunk tag in corpus
    last_correct_type = ''    # type of previously identified chunk tag
    last_guessed = 'O'        # previously identified chunk tag
    last_guessed_type = ''    # type of previous chunk tag in corpus

    for line in iterable:
        line = line.rstrip('\r\n')

        if options.delimiter == ANY_SPACE:
            features = line.split()
        else:
            features = line.split(options.delimiter)

        if num_features is None:
            num_features = len(features)
        elif num_features != len(features) and len(features) != 0:
            raise FormatError('unexpected number of features: %d (%d)' % (len(features), num_features))

        if len(features) == 0 or features[0] == options.boundary:
            features = [options.boundary, 'O', 'O']
        if len(features) < 3:
            raise FormatError('unexpected number of features in line %s' % line)

        guessed, guessed_type = parse_tag(features.pop())
        correct, correct_type = parse_tag(features.pop())
        first_item = features.pop(0)

        if first_item == options.boundary:
            guessed = 'O'

        end_correct = end_of_chunk(last_correct, correct,  last_correct_type, correct_type)
        end_guessed = end_of_chunk(last_guessed, guessed, last_guessed_type, guessed_type)
        start_correct = start_of_chunk(last_correct, correct,  last_correct_type, correct_type)
        start_guessed = start_of_chunk(last_guessed, guessed, last_guessed_type, guessed_type)

        if in_correct:
            if (end_correct and end_guessed and  last_guessed_type == last_correct_type):
                in_correct = False
                counts.correct_chunk += 1
                counts.t_correct_chunk[last_correct_type] += 1
            elif (end_correct != end_guessed or guessed_type != correct_type):
                in_correct = False

        if start_correct and start_guessed and guessed_type == correct_type:
            in_correct = True

        if start_correct:
            counts.found_correct += 1
            counts.t_found_correct[correct_type] += 1
        if start_guessed:
            counts.found_guessed += 1
            counts.t_found_guessed[guessed_type] += 1
        if first_item != options.boundary:
            if correct == guessed and guessed_type == correct_type:
                counts.correct_tags += 1
            counts.token_counter += 1

        last_guessed = guessed
        last_correct = correct
        last_guessed_type = guessed_type
        last_correct_type = correct_type

    if in_correct:
        counts.correct_chunk += 1
        counts.t_correct_chunk[last_correct_type] += 1

    return counts



In [26]:
## 3. conlleval.py
## 3.3 conlleval.py uniq  calculate_metrics metrics



def uniq(iterable):
  seen = set()
  return [i for i in iterable if not (i in seen or seen.add(i))]


def calculate_metrics(correct, guessed, total):
    tp, fp, fn = correct, guessed-correct, total-correct
    p = 0 if tp + fp == 0 else 1.*tp / (tp + fp)
    r = 0 if tp + fn == 0 else 1.*tp / (tp + fn)
    f = 0 if p + r == 0 else 2 * p * r / (p + r)
    return Metrics(tp, fp, fn, p, r, f)


def metrics(counts):
    c = counts
    overall = calculate_metrics( c.correct_chunk, c.found_guessed, c.found_correct)
    by_type = {}
    for t in uniq(list(c.t_found_correct) + list(c.t_found_guessed)):
        by_type[t] = calculate_metrics( c.t_correct_chunk[t], c.t_found_guessed[t], c.t_found_correct[t]  )
    return overall, by_type


In [27]:
## 3. conlleval.py
## 3.4 conlleval.py report  report_notprint


def report(counts, out=None):
    if out is None:
        out = sys.stdout

    overall, by_type = metrics(counts)

    c = counts
    out.write('processed %d tokens with %d phrases; ' %   (c.token_counter, c.found_correct))
    out.write('found: %d phrases; correct: %d.\n' %  (c.found_guessed, c.correct_chunk))

    if c.token_counter > 0:
        out.write('accuracy: %6.2f%%; ' %  (100.*c.correct_tags/c.token_counter))
        out.write('precision: %6.2f%%; ' % (100.*overall.prec))
        out.write('recall: %6.2f%%; ' % (100.*overall.rec))
        out.write('FB1: %6.2f\n' % (100.*overall.fscore))

    for i, m in sorted(by_type.items()):
        out.write('%17s: ' % i)
        out.write('precision: %6.2f%%; ' % (100.*m.prec))
        out.write('recall: %6.2f%%; ' % (100.*m.rec))
        out.write('FB1: %6.2f  %d\n' % (100.*m.fscore, c.t_found_guessed[i]))


def report_notprint(counts, out=None):
    if out is None:
        out = sys.stdout

    overall, by_type = metrics(counts)

    c = counts
    final_report = []
    line = []

    line.append('processed %d tokens with %d phrases; ' % (c.token_counter, c.found_correct))
    line.append('found: %d phrases; correct: %d.\n' % (c.found_guessed, c.correct_chunk))
    final_report.append("".join(line))

    if c.token_counter > 0:
        line = []
        line.append('accuracy: %6.2f%%; ' %  (100.*c.correct_tags/c.token_counter))
        line.append('precision: %6.2f%%; ' % (100.*overall.prec))
        line.append('recall: %6.2f%%; ' % (100.*overall.rec))
        line.append('FB1: %6.2f\n' % (100.*overall.fscore))
        final_report.append("".join(line))

    for i, m in sorted(by_type.items()):
        line = []
        line.append('%17s: ' % i)
        line.append('precision: %6.2f%%; ' % (100.*m.prec))
        line.append('recall: %6.2f%%; ' % (100.*m.rec))
        line.append('FB1: %6.2f  %d\n' % (100.*m.fscore, c.t_found_guessed[i]))
        final_report.append("".join(line))
    return final_report





In [28]:
## 3. conlleval.py
## 3.5 conlleval.py end_of_chunk start_of_chunk


def end_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk ended between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_end = False

    if prev_tag == 'E': chunk_end = True
    if prev_tag == 'S': chunk_end = True

    if prev_tag == 'B' and tag == 'B': chunk_end = True
    if prev_tag == 'B' and tag == 'S': chunk_end = True
    if prev_tag == 'B' and tag == 'O': chunk_end = True
    if prev_tag == 'I' and tag == 'B': chunk_end = True
    if prev_tag == 'I' and tag == 'S': chunk_end = True
    if prev_tag == 'I' and tag == 'O': chunk_end = True

    if prev_tag != 'O' and prev_tag != '.' and prev_type != type_:
        chunk_end = True

    # these chunks are assumed to have length 1
    if prev_tag == ']': chunk_end = True
    if prev_tag == '[': chunk_end = True

    return chunk_end


def start_of_chunk(prev_tag, tag, prev_type, type_):
    # check if a chunk started between the previous and current word
    # arguments: previous and current chunk tags, previous and current types
    chunk_start = False

    if tag == 'B': chunk_start = True
    if tag == 'S': chunk_start = True

    if prev_tag == 'E' and tag == 'E': chunk_start = True
    if prev_tag == 'E' and tag == 'I': chunk_start = True
    if prev_tag == 'S' and tag == 'E': chunk_start = True
    if prev_tag == 'S' and tag == 'I': chunk_start = True
    if prev_tag == 'O' and tag == 'E': chunk_start = True
    if prev_tag == 'O' and tag == 'I': chunk_start = True

    if tag != 'O' and tag != '.' and prev_type != type_:
        chunk_start = True

    # these chunks are assumed to have length 1
    if tag == '[': chunk_start = True
    if tag == ']': chunk_start = True

    return chunk_start



In [29]:
## 3. conlleval.py
## 3.6 conlleval.py return_report main

def return_report(input_file):
    with codecs.open(input_file, "r", "utf8") as f:
        counts = evaluate(f)
    return report_notprint(counts)



In [30]:
### 4. ner.py 
### 4.1 ner.py set_seed  to_list  boolean_string

# set the random seed for repeat
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def to_list(tensor):
    return tensor.detach().cpu().tolist()


def boolean_string(s):
    if s not in {'False', 'True'}:
        raise ValueError('Not a valid boolean string')
    return s == 'True'

In [31]:
### 4. ner.py 
### 4.2 ner.py evaluate


def evaluate(args, data, model, all_ori_tokens):
    model.eval()
    sampler = SequentialSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=args.train_batch_size)

    print("***** Running eval *****")
    print(f" Num examples = {len(data)}")
    print(f" Batch size = {args.eval_batch_size}")

    pred_labels, ori_labels  = [] ,[]

    for b_i, (input_ids, input_mask, segment_ids, label_ids) in enumerate(tqdm_notebook(dataloader, desc="Evaluating")):
        
        input_ids = input_ids.to(args.device)
        input_mask = input_mask.to(args.device)
        segment_ids = segment_ids.to(args.device)
        label_ids = label_ids.to(args.device)

        with torch.no_grad():
            logits = model.predict(input_ids, segment_ids, input_mask)
        # logits = torch.argmax(F.log_softmax(logits, dim=2), dim=2)
        # logits = logits.detach().cpu().numpy()

        for l in logits:
            pred_labels.append([args.id2label[idx] for idx in l])
        
        for l in label_ids:
            ori_labels.append([args.id2label[idx.item()] for idx in l])
    
    eval_list = []
    for ori_tokens, oril, prel in zip(all_ori_tokens, ori_labels, pred_labels):
        for ot, ol, pl in zip(ori_tokens, oril, prel):
            if ot in ["[CLS]", "[SEP]"]:
                continue
            eval_list.append(f"{ot} {ol} {pl}\n")
        eval_list.append("\n")
    
    # eval the model 
    counts = evaluate_(eval_list)
    report(counts)

    # namedtuple('Metrics', 'tp fp fn prec rec fscore')
    overall, by_type = metrics(counts)
    
    return overall, by_type


In [32]:
### 4. ner.py 
## 4.3 instantiate NerProcessor
processor = NerProcessor()

In [33]:
### 4. ner.py 
## 4.4 define label_list, num_labels , id2label


label_list = processor.get_labels(args)

args.num_labels = len(label_list)
args.label_list = label_list

if os.path.exists(os.path.join(args.output_dir, "label2id.pkl")):
  print('label2id.pkl exist, loading:')
  with open(os.path.join(args.output_dir, "label2id.pkl"), "rb") as f:
    label2id = pickle.load(f)
else:
  print('label2id.pkl does not exist, creating')
  label2id = {l:i for i,l in enumerate(label_list)}
  with open(os.path.join(args.output_dir, "label2id.pkl"), "wb") as f:
    pickle.dump(label2id, f)      
    
id2label = {value:key for key,value in label2id.items()} 

print(label2id)
print(id2label)

args.label2id=label2id
args.id2label=id2label

loading labels info from /content/gdrive/My Drive/ETNET/NER/news/code/PeijiYang/checkpoints_bert
label2id.pkl exist, loading:
{'I-EVENT': 0, 'I-LAW': 1, 'I-NORP': 2, 'I-QUANTITY': 3, 'I-CONCEPT': 4, 'I-J': 5, 'B-QUANTITY': 6, 'I-TITLE': 7, 'B-EVENT': 8, 'B-TIME': 9, 'B-FAC': 10, 'I-SLOGAN': 11, 'B-TITLE': 12, 'I-FAC': 13, 'I-IDIOM': 14, 'B-PERSON': 15, 'B-TERM': 16, 'I-TERM': 17, 'I-LOC': 18, 'I-POLICY': 19, 'I-PRODUCT': 20, 'I-ORG': 21, 'B-J': 22, 'B-ENGLISH': 23, 'O': 24, 'B-FIN': 25, 'B-PRODUCT': 26, 'I-WORK_OF_ART': 27, 'B-POLICY': 28, 'I-TIME': 29, 'B-UNIT': 30, 'B-LANGUAGE': 31, 'I-LANGUAGE': 32, 'B-WORK_OF_ART': 33, 'I-FIN': 34, 'B-LOC': 35, 'B-IDIOM': 36, 'B-ORG': 37, 'I-PERSON': 38, 'B-LAW': 39, 'I-ENGLISH': 40, 'B-SLOGAN': 41, 'B-NORP': 42, 'B-CONCEPT': 43, 'I-UNIT': 44}
{0: 'I-EVENT', 1: 'I-LAW', 2: 'I-NORP', 3: 'I-QUANTITY', 4: 'I-CONCEPT', 5: 'I-J', 6: 'B-QUANTITY', 7: 'I-TITLE', 8: 'B-EVENT', 9: 'B-TIME', 10: 'B-FAC', 11: 'I-SLOGAN', 12: 'B-TITLE', 13: 'I-FAC', 14: 'I-IDI

In [34]:
### load bert config, tokenizer and models
print('loading Config')
config = BertConfig.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,     
                                    num_labels=args.num_labels,
                                    id2label=args.id2label, label2id=args.label2id,
                                    cache_dir=args.cache_dir,
                                    )##  


print('loading tokenizer')
tokenizer = BertTokenizer.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,                                            
                                          do_lower_case=args.do_lower_case,
                                          cache_dir=args.cache_dir,
                                          use_fast=False,)

  
  
print('loading model')
model = BERT_BiLSTM_CRF.from_pretrained(args.model_name_or_path, config=config,  need_birnn=args.need_birnn, rnn_dim=args.rnn_dim)
print('loading model to device:',device)
model.to(device)

loading Config
loading tokenizer
loading model


Some weights of the model checkpoint at /content/gdrive/My Drive/ETNET/NER/news/code/PeijiYang/pretrain_models/bert-base-multilingual-cased were not used when initializing BERT_BiLSTM_CRF: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BERT_BiLSTM_CRF from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BERT_BiLSTM_CRF from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BERT_BiLSTM_CRF were not initialized from the model checkpoint at /content/gdrive/My Drive/ETNET/NER/news/code/PeijiYang/pretrain_models/bert-base-multilingual-cased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.de

loading model to device: cuda


BERT_BiLSTM_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [35]:
train_examples, train_features, train_data = get_Dataset(args, processor, tokenizer, mode="train")
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)
print(len(train_examples), len(train_features), len(train_data))

if args.do_eval:
    eval_examples, eval_features, eval_data = get_Dataset(args, processor, tokenizer, mode="eval")
    print(len(eval_examples), len(eval_features), len(eval_data))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


convert examples: 69it [00:00, 685.42it/s]


*** Example ***
guid: 0
tokens: [CLS] 投 資 者 或 會 對 近 期 的 經 歷 感 到 非 常 恐 慌 ， 因 此 他 們 在 一 段 長 時 間 內 遠 離 股 市 及 傾 向 持 有 現 金 的 可 能 性 確 實 存 在 。 [SEP] **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL**
input_ids: 101 4037 7442 6457 3978 4459 3457 7697 4470 5718 6228 4802 3911 2555 8332 3626 3834 3921 10064 3000 4792 2196 2335 3031 2072 4823 8127 4388 8137 2447 776

convert examples: 158472it [03:51, 685.61it/s]


158472 158472 158472


convert examples: 0it [00:00, ?it/s]


*** Example ***
guid: 0
tokens: [CLS] 科 技 股 大 反 彈 ， 深 成 指 急 漲 2 . 6 6 % ， 創 業 板 指 更 飆 3 . 2 5 % 。 [SEP] **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL**
input_ids: 101 5954 4030 6510 3

convert examples: 27966it [00:39, 699.60it/s]


27966 27966 27966


In [36]:
 # Prepare optimizer and schedule (linear warmup and decay)

t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [ {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                  {'params': [p for n, p in model.named_parameters() if    any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps = t_total)

model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level, loss_scale="dynamic")

##optimizer = FusedAdam(optimizer_grouped_parameters,  lr=args.learning_rate,  bias_correction=False, )#

#scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(1 + 0.05*epoch))
#if args.loss_scale == 0:
#   optimizer = FP16_Optimizer(optimizer, dynamic_loss_scale=True)
#else:
#   optimizer = FP16_Optimizer(optimizer, static_loss_scale=args.loss_scale)

print("\n\n")
print("***** Running training *****")
print("  Num examples = %d", len(train_data))
print("  Num Epochs = %d", args.num_train_epochs)
print("  Total optimization steps = %d", t_total)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic



***** Running training *****
  Num examples = %d 158472
  Num Epochs = %d 50.0
  Total optimization steps = %d 60050.0


In [ ]:
args.clip_grad=2
model.train()

global_step = 0
tr_loss, logging_loss, best_f1 = 0.0, 0.0,  0.0



for ep in trange(int(args.num_train_epochs), desc="Epoch"):
    model.train()
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):

      batch = tuple(t.to(device) for t in batch)
      input_ids, input_mask, segment_ids, label_ids = batch
      outputs = model(input_ids, label_ids, segment_ids, input_mask)
      loss = outputs

      if n_gpu > 1:
        loss = loss.mean() # mean() to average on multi-gpu.
      if args.gradient_accumulation_steps > 1:
        loss = loss / args.gradient_accumulation_steps


     
      with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()

      nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)

      tr_loss += loss.item()
      


      if (step + 1) % args.gradient_accumulation_steps == 0:
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        global_step += 1
 

        if args.logging_steps > 0 and global_step % args.logging_steps == 0:
          tr_loss_avg = (tr_loss-logging_loss)/args.logging_steps
          writer.add_scalar("Train/loss", tr_loss_avg, global_step)
          logging_loss = tr_loss
            
    if args.do_eval:
      all_ori_tokens_eval = [f.ori_tokens for f in eval_features]
      overall, by_type = evaluate(args, eval_data, model,  all_ori_tokens_eval)
                
      # add eval result to tensorboard
      f1_score = overall.fscore
      writer.add_scalar("Eval/precision", overall.prec, ep)
      writer.add_scalar("Eval/recall", overall.rec, ep)
      writer.add_scalar("Eval/f1_score", overall.fscore, ep)

      # save the best performs model
      if f1_score > best_f1:
        print(f"----------the best f1 is {f1_score}---------")
        best_f1 = f1_score
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

        print(f'epoch {ep}, train loss: {tr_loss}')
        # writer.add_graph(model)
writer.close()

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 256.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 128.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 64.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16.0

***** Running eval *****
 Num examples = 27966
 Batch size = 132


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


...

epoch 2, train loss: 7837963.610229492



***** Running eval *****
 Num examples = 27966
 Batch size = 132



processed 1264735 tokens with 157228 phrases; found: 161591 phrases; correct: 143950.
accuracy:  96.20%; precision:  89.08%; recall:  91.55%; FB1:  90.30
          CONCEPT: precision:  86.60%; recall:  87.97%; FB1:  87.28  1224
          ENGLISH: precision:  30.77%; recall:  25.64%; FB1:  27.97  65
            EVENT: precision:  89.99%; recall:  92.91%; FB1:  91.43  13251
              FAC: precision:  83.27%; recall:  86.55%; FB1:  84.88  5895
              FIN: precision:  89.49%; recall:  90.54%; FB1:  90.01  14551
            IDIOM: precision:  79.41%; recall:  84.76%; FB1:  82.00  1156
                J: precision:  94.23%; recall:  93.31%; FB1:  93.76  7413
         LANGUAGE: precision:  69.57%; recall:  64.00%; FB1:  66.67  23
              LAW: precision:  80.62%; recall:  86.22%; FB1:  83.33  6063
              LOC: precision:  92.05%; recall:  92.08%; FB1:  92.06  13975
             NORP: precision:  87.89%; recall:  92.47%; FB1:  90.12  545
              ORG: precision:  88

Epoch:   8%|▊         | 4/50 [1:48:24<20:47:44, 1627.48s/it]

epoch 3, train loss: 8750944.11843872


In [ ]:
### 4. ner.py 
### 4.1 ner.py set_seed  to_list  boolean_string